In [4]:
from openai import OpenAI
import pandas as pd
import numpy as np
import re
import os

In [5]:
path = "datasets"
filename = "original.csv"

ddos_data = pd.read_csv("./" + path + "/" + filename)

In [6]:
# Complete your model name / apikey / url

llm_model = "deepseek-chat"
client = OpenAI(
    api_key="sk-aaaa1111bbbb2222cccc",
    base_url="https://api.deepseek.com"
)


In [7]:
ddos_data_without_label = ddos_data.drop([" Label"], axis=1)
# print(ddos_data_without_label.max())

In [8]:
# calculate the max, min, median, mean and variance
max_values = ddos_data_without_label.max()
min_values = ddos_data_without_label.min()
median_values = ddos_data_without_label.median()
mean_values = ddos_data_without_label.mean()
variance_values = ddos_data_without_label.var()

# 将结果合并成一个新的 DataFrame
result = pd.DataFrame({
    'Max': max_values,
    'Min': min_values,
    'Median': median_values,
    'Mean': mean_values,
    'Variance': variance_values
})

In [ ]:
# Generate the Knowledge Prompt

know_prompt_pre = f"Supposed that you are now an experienced network traffic data analysis expert. You need to help me analyze the data in the DDoS dataset and determine whether the data is DDoS traffic or normal traffic. Next, I will give you the maximum, minimum, median, mean, and variance of all the data under each label in the data set, which may help you make your judgment. "

know_prompt_back = ""

for i in range(result.shape[0]):
    know_prompt_back += f"{result.index[i]}'s max value is{result.iloc[i].iloc[0]:0.1f}, min value is{result.iloc[i].iloc[1]:0.1f}, median value is{result.iloc[i].iloc[2]:0.1f}, mean value is{result.iloc[i].iloc[3]:0.1f}, variance value is{result.iloc[i].iloc[4]:0.1f},"

know_prompt = know_prompt_pre + know_prompt_back + ". Just remember this knowledge and you don’t need to answer me anything."

print(know_prompt)

In [10]:
# regex pattern
pattern = r'\[(.*?)\]'

In [ ]:
# role reasoning inference mode

# p0 = bp
# p1 = bp + cod
# p2 = bp + cod + cot
# p31 = kp + bp + cod
# p3 = kp + bp + cod + cot

role_mode = 0

p0, p1, p2, p31, p3 = False, False, False, False, False
if role_mode == 0:
    p0, p1, p2, p31, p3 = True, False, False, False, False
elif role_mode == 1:
    p0, p1, p2, p31, p3 = False, True, False, False, False
elif role_mode == 2:
    p0, p1, p2, p31, p3 = False, False, True, False, False
elif role_mode == 3:
    p0, p1, p2, p31, p3 = False, False, False, True, False
elif role_mode == 4:
    p0, p1, p2, p31, p3 = False, False, False, False, True

print(p0, p1, p2, p31, p3)

In [29]:
precount = 0
file_path = f"./result/deepseek_{role_mode}.csv"

In [ ]:
# 增量式保存 确定上一个起始点和保存的文件位置
startwith = 0
if os.path.isfile(file_path):
    test = pd.read_csv(file_path)
    startwith = test.shape[0]

print("startwith is: ", startwith)


# 创建一个空的 DataFrame，保存最终的结果
predict_df = pd.DataFrame(columns = ["index", "attack", "benign", "original"])

token_prompt_pre = ""
if role_mode == 0:
    # bp
    token_prompt_pre = f"Next, I will give you a piece of data about network traffic information. You need to first tell me the probability of this data belonging to DDoS traffic data and normal traffic data directly and separately, and express the probability in the format of [0.xxx,0.xxx]. The characteristics of its network traffic data packets are"
elif role_mode == 1:
    # bp + cod
    token_prompt_pre = f"Next, I will give you a piece of data about network traffic information. You need to first tell me the probability of this data belonging to DDoS traffic data and normal traffic data directly and separately, and express the probability in the format of [0.xxx,0.xxx]. The first number is the probability of DDoS traffic, and the second number is the probability of normal traffic. Both numbers are decimals between 0 and 1, and the sum of the two numbers is 1. The characteristics of its network traffic data packets are"
elif role_mode == 2 or role_mode == 4:
    # bp + cod + cot
    # kp + bp + cod + cot
    token_prompt_pre = f"Next, I will give you a piece of data about network traffic information. You need to first tell me the probability of this data belonging to DDoS traffic data and normal traffic data directly and separately, and express the probability in the format of [0.xxx,0.xxx]. The first number is the probability of DDoS traffic, and the second number is the probability of normal traffic. Both numbers are decimals between 0 and 1, and the sum of the two numbers is 1. Let's think step by step and explain the reasons for your judgment. The characteristics of its network traffic data packets are"
elif role_mode == 3:
    token_prompt_pre = f"Next, I will give you a piece of data about network traffic information. You need to first tell me the probability of this data belonging to DDoS traffic data and normal traffic data directly and separately, and express the probability in the format of [0.xxx,0.xxx]. The first number is the probability of DDoS traffic, and the second number is the probability of normal traffic. Both numbers are decimals between 0 and 1, and the sum of the two numbers is 1. The characteristics of its network traffic data packets are"

print("token_prompt_pre is: ", token_prompt_pre)

# print
for i in range(startwith, ddos_data.shape[0]):
    token_prompt_back = ""

    # -1的原因是去掉Label
    netlabels = ddos_data.shape[1] - 1
    for j in range(netlabels):
        if j != netlabels-1:
            token_prompt_back += f"{ddos_data.columns[j]}:{ddos_data.iloc[i].iloc[j]},"
        else:
            token_prompt_back += f"{ddos_data.columns[j]}:{ddos_data.iloc[i].iloc[j]}"
    
    token_prompt = token_prompt_pre + token_prompt_back

    # non-streaming
    messages = []
    if role_mode == 3 or role_mode == 4:
        messages = [{'role': 'user', 'content': know_prompt},
                    {'role': 'user', 'content': token_prompt}]   
    else: 
        messages = [{'role': 'user', 'content': token_prompt}]


    completion = client.chat.completions.create(model=llm_model, messages=messages)
    
    matches = re.findall(pattern, completion.choices[0].message.content)
    if len(matches) != 0:
        result = matches[0].split(",")

        # print(pattern, completion.choices[0].message.content)
    
        if len(result) == 2:
            print(f"| {i} | {result[0]}, {result[1]} | {matches[0]} |")
            # save to dataframe
            predict_df.loc[i] = [i, result[0], result[1], matches[0]]
        else:
            print(f"| {i} | None, None | {matches} |")
            # save to dataframe
            predict_df.loc[i] = [i, "None", "None", completion.choices[0].message.content]
    else:
        print(f"| {i} | None, None | {matches} |")
        # save to dataframe
        predict_df.loc[i] = [i, "None", "None", completion.choices[0].message.content]

    # for debug
    if i == startwith + precount - 1:
        break

print("----- ----- done ----- -----")

In [31]:
# 增量式保存：如果文件存在则追加，不存在则创建
if not os.path.isfile(file_path):
    predict_df.to_csv(file_path, index=False)
else:
    predict_df.to_csv(file_path, mode='a', header=False, index=False)